# V1 Units Dashboard — Pure JupyterLite

This dashboard runs **entirely in your browser** (Pyodide).  
No server required. Code cells are hidden by default.

In [ ]:
# Hide inputs (you can toggle by setting SHOW_CODE=True and re-running this cell)
from IPython.display import HTML, display
SHOW_CODE = False
_style_id = "hide-code-style"

def _set_code_visibility(show: bool):
    css = "" if show else "div.jp-Cell-inputWrapper { display: none; }"
    html = f"<style id='{_style_id}'>{css}</style>"
    display(HTML(html))

_set_code_visibility(SHOW_CODE)

In [ ]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as W
from IPython.display import display, clear_output
import utils as U

UNITS, CTRL, LASER, THETAS = U.synth_dataset(n_units=400, seed=1, ori_step=10)

# Widgets
layer_opts = ["SG", "G", "IG"]
layer_sel  = W.SelectMultiple(options=layer_opts, value=tuple(layer_opts), description="Layers", rows=3)
osi_rng    = W.FloatRangeSlider(value=[0.0, 1.0], min=0.0, max=1.0, step=0.01, description="OSI")
hbw_rng    = W.IntRangeSlider(value=[5, 90], min=1, max=180, step=1, description="HBW (°)")
effect_dd  = W.Dropdown(options=["Any", "MUL", "MXH"], value="Any", description="Effect")
show_mean  = W.Checkbox(value=True, description="Show mean curve")
unit_pick  = W.Dropdown(options=[("—", None)], value=None, description="Unit")

scatter_out = W.Output(layout=W.Layout(border="1px solid #e0e0e0", min_height="280px"))
tuning_out  = W.Output(layout=W.Layout(border="1px solid #e0e0e0", min_height="360px"))
detail_out  = W.Output(layout=W.Layout(border="1px solid #e0e0e0", min_height="360px"))
status_html = W.HTML()

def refresh(*_):
    df = U.filter_units(
        UNITS,
        layers=list(layer_sel.value),
        osi_range=tuple(osi_rng.value),
        hbw_range=tuple(hbw_rng.value),
        effect=effect_dd.value
    )

    # Update unit picker
    if len(df):
        unit_pick.options = [(f"Unit {int(u)} ({r.layer}, {r.effect})", int(u)) for u, r in df.set_index("unit_id").iterrows()]
        unit_pick.value = unit_pick.options[0][1]
    else:
        unit_pick.options, unit_pick.value = [("—", None)], None

    # Status
    status_html.value = f"<b>Matched units:</b> {len(df)}"

    # Scatter
    with scatter_out:
        clear_output()
        fig, ax = plt.subplots(figsize=(5, 3))
        ax.scatter(df["HBW"], df["OSI"], alpha=0.7)
        ax.set_xlabel("HBW (deg)"); ax.set_ylabel("OSI"); ax.set_title("OSI vs HBW (filtered)")
        ax.grid(True, alpha=0.3)
        display(fig)

    # Overlay tuning
    with tuning_out:
        clear_output()
        fig, ax = plt.subplots(figsize=(5.5, 3))
        U.overlay_plot(ax, ctrl=CTRL, laser=LASER, thetas=THETAS, unit_ids=df["unit_id"].tolist(), show_mean=show_mean.value)
        display(fig)

    # Detail for selected unit
    show_unit_detail()

def show_unit_detail(*_):
    uid = unit_pick.value
    with detail_out:
        clear_output()
        if uid is None:
            print("Select a unit to view details.")
            return
        row = UNITS.loc[UNITS.unit_id == uid].iloc[0]
        yc = CTRL.loc[CTRL.unit_id == uid, "rate"].values
        yl = LASER.loc[LASER.unit_id == uid, "rate"].values
        fig, ax = plt.subplots(figsize=(5.5, 3))
        ax.plot(THETAS, yc, label="Control")
        ax.plot(THETAS, yl, label="Laser", linestyle="--")
        ax.set_title(f"Unit {uid} — layer={row.layer}, effect={row.effect}, OSI={row.OSI:.2f}, HBW={row.HBW:.1f}°")
        ax.set_xlabel("Orientation (deg)"); ax.set_ylabel("FR (a.u.)")
        ax.legend(loc="best"); ax.grid(True, alpha=0.3)
        display(fig)

# Wire callbacks
for w in (layer_sel, osi_rng, hbw_rng, effect_dd, show_mean):
    w.observe(refresh, names="value")
unit_pick.observe(show_unit_detail, names="value")

# Initial render
refresh()

controls = W.VBox([
    W.HTML("<b>Filters</b>"),
    layer_sel,
    osi_rng,
    hbw_rng,
    effect_dd,
    show_mean,
    W.HTML("<hr><b>Inspect</b>"),
    unit_pick,
    W.HTML("<hr>")
], layout=W.Layout(min_width="280px"))

layout = W.HBox([controls, W.VBox([status_html, scatter_out, tuning_out, detail_out])])
display(layout)